In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

In [ ]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'train.csv', header=None) # re-name test.csv 
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})

# display 
test_df.head()

In [ ]:
!pip install allennlp==1.0.0 allennlp-models==1.0.0

!pip install nltk
!pip install tqdm

!pip install -U spacy 
!python -m spacy download en_core_web_sm

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")

In [ ]:
prediction = predictor.predict(
        sentence = "TOP TWEETS -- @kurtbardella: if the first four weeks of the #Obama presidency were like this when I worked at #oversight we'd have issued so many subpoenas by now ..."
)
prediction

START AGAIN

In [ ]:
def extract_modal(obj):
    tags = list() 
    parts = list()
    words = list()
    nouns = list()
    verbs = list()

    includes= list()
    cutouts = list()
    conseqs = list()
    anteced = list()

    def extract(obj):

      if isinstance(obj, dict):
        for k, v in obj.items():
          if k=='nodeType':
            tags.append(v)
          if k=='word':
            words.append(v)
          #if k=='children': # as below
          if isinstance(v, (dict, list)):
            extract(v)
        
      elif isinstance(obj, list):
        for item in obj:
          if isinstance(item, dict):
            extract(item)

            if len(words)>0:
              #print(words)
              zoom = 0
              index=-1
              if 'SBAR' in tags: 
                index = max(idx for idx, val in enumerate(tags) if val=='SBAR')
                zoom = index
              if 'S' in tags:
                index = max(idx for idx, val in enumerate(tags) if val=='S')
                if index > zoom:
                  zoom = index
              head = words[zoom]
              #print(tags, zoom, head)
              word = item['word']
              if (
                  'would'==word.lower() or 'wouldn'==word.lower() or 'wouldn\'t'==word.lower() or 'woulnd\'t'==word.lower() or 
                  'could'==word.lower() or 'could\'t'==word.lower() or 'coulnd\'t'==word.lower() or
                  'might'==word.lower() or 'mightn\'t'==word.lower() or 'mighn\'t'==word.lower() or 'ought'==word.lower() or  
                  'should'==word.lower() or 'shouldn\'t'==word.lower() or 'shoulnd\'t'==word.lower() or 
                  'd'==word.lower() or word.lower().strip().endswith('\'d') 
              ):
                #print('word:',word)
                if word not in head: # without .lower()
                  head = scrap_from(obj,item)
                parts.append({'tag':'MD','word':head,'text':obj})
                includes.append(word)
      
              if 'VBP' in tags: # 'MD' misspelled (and hence mistagged) as 'VBP'
                if (
                    word.lower().startswith('wouln') or word.lower().startswith('couln') or
                    word.lower().startswith('shouln')
                ):     
                  parts.append({'tag':'MD','word':head,'text':obj}) # treat 'VBP' as 'MD'
                  includes.append(word)

              if 'NP' in tags: # for debugging only!
                #print('NP:', word)
                #print('head:',head)
                nouns.append({'tag':'NP','word':word,'text':obj})

              if 'SINV' in tags: # Inverted declarative sentence (eg Had it been...)
                parts.append({'tag':'SINV','word':head,'text':obj}) 

              if 'VP' in tags:
                #print('VP:', word)
                verbs.append({'tag':'VP','word':word,'text':obj})
                if 'wish' in word.lower() or 'shoul' in word.lower():
                  if word not in head:
                    head = scrap_from(obj,item)
                  parts.append({'tag':'VP','word':head,'text':obj})
                  includes.append(word)

              if 'VBD' in tags: 
                #print('VBD:', word)
                if 'were'==word.lower() or 'was'==word.lower() or 'had'==word.lower():
                  #print('head:',head)
                  if word not in head:
                    head = scrap_from(obj,item)
                  parts.append({'tag':'VBD','word':head,'text':obj})
                  includes.append(word)
                
              if 'VBN' in tags: # parser mistakes 'VBN' for 'VBD'(past) 
                if 'had'==word.lower():
                  if word not in head:
                    head = scrap_from(obj,item)
                  parts.append({'tag':'VBN','word':head,'text':obj})
                  includes.append(word) 

              if 'RB' in tags: 
                if 'even'==word.lower():
                  #if word not in head:
                  #  head = scrap_from(obj,item)
                  parts.append({'tag':'MD','word':head,'text':obj}) # treat 'RB' as 'MD'
                  includes.append(word)
                      
              if 'IN' in tags: 
                if (
                    'if' == word.lower() or #'then' == word.lower() or
                    'with'==word.lower() or 'without'==word.lower()
                ): 
                  if word not in head:
                    #print('head:',head)
                    #print('IN:', word)
                    head = scrap_from(obj,item)
                  parts.append({'tag':'IN','word':head,'text':obj})
                  includes.append(word)     

              if 'CC' in tags:
                if (
                    'but' == word.lower()
                ):
                  if word not in head:
                    head = scrap_from(obj,item)[3] # remove 'but'
                    #print(head)
                  parts.append({'tag':'CC','word':head,'text':obj})
                  includes.append(word)

              tag = tags[-1]
              phrase = words[-1]
              if 'S'==tag or 'CC'==tag:
                cut = True
                for include in includes:
                  if include in phrase:
                    cut = False
                if cut and phrase not in cutouts:
                  cutouts.append(phrase)
        return

    extract(obj)
 
    #print(cutouts)
    for part in parts:
      tag = part['tag']
      text = part['text'] 
      clause= part['word']    
 
      tockenized = word_tokenize(clause)
      tokens = set(tockenized)
      if len(tokens) > 1:
        #print(tag,':',clause)
        if part['tag'] == 'MD':
          exclude = False
          for consequent in conseqs:
            if clause in consequent or consequent in clause:
              exclude = True
          if not exclude and clause not in conseqs: 
            conseqs.append(clause)
        if part['tag'] in 'SINV VBD VBN VP IN RB CC':
          exclude = False
          for consequent in conseqs:
            if consequent in clause:
              exclude = True
          if exclude and (clause in conseqs or consequent in clause) and len(conseqs)>1:
            if clause in conseqs:
              conseqs.remove(clause)    
          for antecedent in anteced:
            if clause in antecedent or antecedent in clause:
              exclude = True
          if not exclude and clause not in anteced:      
            anteced.append(clause)

    return conseqs, anteced, nouns, verbs

In [ ]:
def scrap_from(obj, item=None): 
  word = str() 
  prev = str() 
  for h in obj: # head word
    if item==None or h['word']==item['word']: 
      word = ' ' 
    if len(word) > 0: 
      word = word.strip() 
      if (not (
              word.lower()=='\'m' or word.lower()=='\'ll' or
              word.lower()=='\'s' or word.lower()=='d' or # I'd be...
              word.lower()=='\'re' or word.lower()=='\'ve' or word.lower()=='n\'t') and  
          not (prev.lower().endswith('s') and word.lower()=='\' ') and # before\':peoples'
          not (word.lower() in '+.: m k x' and prev[-1:].isnumeric()) and # eg:'10:30am', '3M'
          not (prev.lower() in '+' and word.isnumeric()) and
          not (
              prev.lower() in '¥$€£&@({[/---#'  or  # after(eg 'S& P')
              word.lower() in '^~*:&%)}]/---...,;' # before(eg '% 5 M')
          )
      ): 
        word = word + ' '
      word = word + h['word']  
      prev = word
  word = normalize(word)     
  #print(word)    
  return word

In [ ]:
def separate_children(obj):
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [ ]:
def extract_constituents(prediction):
 for k, v in prediction.items():
  if k=='hierplane_tree':
    for ke, val in v.items():
      if ke=='root':
        for key, value in val.items():
          if key == 'children':
            return extract_modal(value)  

In [ ]:
from nltk.tokenize import word_tokenize

def simplify(constit):
  if isinstance(constit, str):
    word = normalize(constit)
    return word
   
  constit = constit['text']

  gist = list();
  unique = set() 
  for c in constit:
      p = separate_children(c) # list of c['word']s
      for head in sorted(p,key=len):
        #print(head) # head phrase (few words or one char ',')
        tokenized = word_tokenize(head)
        tokens = set(tokenized) #  without duplicates (eg ',')
        if len(unique.intersection(tokens)) <= len(tokens):
          unique.update(tokens) # without duplicate tokens!!
          gist.extend(tokenized)
  #print(gist)     
  result = str()
  prev = str()
  for word in gist:
    word = word.strip()
    if (
        not (
             word.lower()=='\'m' or word.lower()=='\'ll' or 
             word.lower()=='\'s' or word.lower()=='d' or # I'd be doing...
             word.lower()=='\'re' or word.lower()=='\'ve' or word=='n\'t'
        ) and
        not (prev.lower().endswith('s') and word.lower()=='\' ') and # before\':peoples'
        not (word.lower() in '+.: m k x' and prev[-1:].isnumeric()) and # after '10:30am'
        not (prev.lower() in '+' and word.isnumeric()) and
        not (prev.lower()=='i' and word.lower()=='d') and # ID - identification
        not  
        (
          prev.lower() in '¥$€£&@({[/---#' or  # ' ' after
          word.lower() in '^~*:&%)}]/---...,;'# or before
        )
    ): 
      result = result + ' '
      #print(prev,'-->',word)
    prev = word
    result = result + word 
  result = normalize(result)   
  #print(phrase,':',result)  
  return result

In [ ]:
import re
regex = '\\\'\s+([^\']+)\s+\\\''
def compact(fragment):
# specify the number of replacements by changing the 4th argument
  result = re.sub(regex, '\''+r'\1'+'\'' , fragment)
  return result

In [ ]:
def normalize(fragment):
  r = compact(fragment).strip()
  r = r.replace('   ',' ') 
  r = r.replace('  ',' ')
  r = r.replace(' ,',',') 
  r = r.replace(' .','.')
  r = r.replace('Mr.','Mr. ')
  r = r.replace('Ms.','Ms. ')
  r = r.replace('Mrs.','Mrs. ')
  r = r.replace('.  ','. ') # Mr./Ms./Mrs.
  r = r.replace('... ','...')
  r = r.replace(' --- ','---')
  r = r.replace(' -- ','--')
  r = r.replace(' - ','-')
  r = r.replace(' !','!')
  r = r.replace(' ?','?')
  r = r.replace('$ ','$')
  r = r.replace('# ','#')
  r = r.replace(' %','%')
  r = r.replace('( ','(')
  r = r.replace(' )',')')
  r = r.replace(' ;',';')
  r = r.replace(' :',':') 
  r = r.replace(': ',':') #50/50 - carefull!
  r = r.replace(' nt','nt')
  r = r.replace(' (k)','(k)') #401(k)
  r = r.replace(' (tm)','(tm)')
  r = r.replace('n na ','nna ')
  r = r.replace(' \'m ','\'m ')
  r = r.replace('s \' ','s\' ') # was 'neurtralized'?
  r = r.replace(' \'s ','\'s ') # a 'surgical' strike!
  r = r.replace(' \'d ','\'d ')
  r = r.replace(' \'re ','\'re ')
  r = r.replace(' \'ve ','\'ve ')
  r = r.replace(' \'ll ','\'ll ')
  r = r.replace(' n\'t','n\'t') # what about UPPER()?
  r = r.replace(' \' ',' \'')
  r = r.rstrip()
  return r

In [ ]:
def defragment(fragments, text, original, removed):
  result = []
  candid = []
  defrag = []
  prev = str() # previous fragment
  start = 0 # start of next fragment
  end = -1 # end of previous fragment  
  for f in sorted(fragments,key=len):
    r = normalize(f)
    #print(prev+' | '+r)
    try: 
      start = text.index(r)
    except ValueError:
      #print('\n{'+r+'} not in:\n '+text) # this is now in offset()
      pass
    if start > end: # assume no overlap with prev (ie separte fragment)
      defrag = prev + r # try to join two fragments
      index = text.find(defrag) # is it exactly consequitive fragment?
      if index < 0 or index > end: # it separate and non-consequitive 
        end = start + len(r) - 1
        prev = r
        if end < len(text):
          candid.append(overlay(start, {'labels':0, 'text':r, 'start':start, 'end':end}, original, removed))
      elif index > 0 and index+len(prev)+len(r) < len(text): # consequitive
        start = index 
        end = start+len(prev)+len(r)-1
        prev = text[start:end]
        candid.append(overlay(start, {'labels':0, 'text':defrag, 'start':start, 'end':end}, original, removed))
    elif start < end: # next fragment is located before prev end
      defrag = r + prev
      index = text.find(defrag) # is it exactly preceding prev fragment?
      #print(text,index,defrag)
      if index > 0 and index+len(prev)+len(r)-1 == end:
        start = index
        prev = text[start:end]
        candid.append(overlay(start, {'labels':0, 'text':defrag, 'start':start, 'end':end}, original, removed))
      elif start >= 0:
        end = start + len(r) - 1
        prev = r
        if end < len(text):
          candid.append(overlay(start, {'labels':0, 'text':r, 'start':start, 'end':end}, original, removed))
        
  if len(candid) == 1:
    candid[0].update(labels = 1) # gold  

  result.extend(candid)

  return result       

In [ ]:
def overlay(starting, fragment, original, removed):
  assert isinstance(fragment, dict)

  base = offset(fragment['text'], starting, original)
  text = base[0]
  start= base[1]
  end = base[2]
  #print(start,text,end)

  for r in removed:
    #print(r[0],start,r[1],end)
    if r[0] <= start:
      start=start + 1
      end = end + 1
      #print(start,text,end)
    elif r[0] <= end:
      end = end + 1
      #print(start,text,end) 
    
  fragment.update(text = text)  
  fragment.update(start=start) 
  fragment.update(end = end) 

  return fragment

In [ ]:
def offset(text, starting, original):
  start = -1
  end = -1
  prev = str()
  for c in text:
    try: 
      #print(c, start, prev, end)
      start = original.index(prev + c, starting)
      end = start + len(prev) + len(c) - 1
    except ValueError:
      if c==' ': # eg.'Pfizer (NYSE: '+'PFE)'
        last = original.find(prev, starting)
        if last > -1: # back 1 space helped
          prev = prev # + c
          continue    # skip 1 space
        else:   
          if end >= 0: # reverse from end
            end = end + 1 # 1 space
            start = end - len(prev)
            text = original[start:end]
            #print(start, prev, end, text)
          else:
            print('\n\"'+prev+'\" not from '+str(starting) + ' in:'+original)
          return text, start, end  
      else: 
        last = original.find(prev + ' ' + c, starting)
        if last > -1: # add 1 space 
          prev = prev + ' ' + c
          continue
        else:  
          last = original.find(prev+c)
          if last == -1: # eg 'bla-' instead of 'bla -'
            last = original.find(prev) # leave 'bla'
          if last > -1: # without space!
            start = last
            end = start + len (prev) + len(c) 
            prev = prev + c
            continue
          if start >= 0: # reverse from start
            end = start + len(prev) + len(c)
            text = original[start:end]
            #print(c,start,prev,end,text)
          else:
            print('\n\"'+prev+c+'\" not from '+str(starting) + ' in:'+original)  
          return text,start,end 

    prev = prev + c

  #print(start, prev, end, text)  
  return text, start, end  

In [ ]:
def record(original):
  base=[(i, c) for i, c in enumerate(original)]
  return base

In [ ]:
def contract(text, base):
  removed = []
  i=0
  for b in base: 
    if text.find(b[1],b[0]-i)==-1:
      removed.append(b)
      i = i + 1
  return removed

In [ ]:
def separate_from(md,pa):
  if pa in md:
    if md.index(pa) > 0:
      md = md[0:md.index(pa)]
      #print('|'+md+'{...}')
    else:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md        

In [ ]:
def separate(modal,past):
  for md in sorted(modal,key=len):
    for pa in reversed(sorted(past,key=len)):
      #print(md,'|',pa)
      new = separate_from(md, pa)
      if new != md:
        if md in modal: # remove original
          modal.remove(md)
        if len(new)>0 and new not in modal:
          modal.append(new) 
        return # only once - for longest          

In [ ]:
def chunk(original):
  verbs = list()
  modal = list()
  nounm = list()
  past = list() 
  nounp = list()
  nouns = list()

  base = record(original)
  original = original.replace('\"','')
  #original = original.replace('#','')
  removed = contract(original, base)
  
  text = normalize(original) # remove extra spaces

  prediction = predictor.predict(sentence=text)
  mod, pas, nou, ver = extract_constituents(prediction)
 
  for v in ver:
    vr = simplify(v) 
    if vr not in verbs:
      verbs.append(vr)

  for u in nou:
    nu = simplify(u) 
    if nu not in nouns:
      nouns.append(nu) 

  if len(mod)==0: # if no modal('MD') detected 
    for p in pas:
      pa = simplify(p)
      if pa not in past:
          past.append(pa) 

  for m in reversed(sorted(mod,key=len)):
    md = simplify(m) # get one string gist
    if len(md)==0:
      continue
    #print('md: '+md)
    
    for nm in reversed(sorted(nouns,key=len)):  
      if nm in md:   
        nounm.append(nm)
        nouns.remove(nm)

    for p in sorted(pas,key=len):
      pa = simplify(p)
      if len(pa)==0:
        continue
      #print('pa: ',pa)

      for np in reversed(sorted(nouns,key=len)):
        if np in pa:
          nounp.append(np)
          nouns.remove(np)

      if pa not in past and len(pa) > 0:
        past.append(pa)         

    if md not in modal and len(md) > 0:        
      modal.append(md)

  separate(modal, past)   

  # get start, end positions 
  modals = defragment(modal, text, original, removed)
  #if len(modals)==0: # rules: consequent can be empty
  #  modals = defragment(verbs, text, original, removed)

  pasts = defragment(past, text, original, removed)
  if len(pasts)==0: # because antecedent can't be empty
    if len(modals) > 0:
      pasts.append(modals.pop(0)) # why 0? can be any!
      pasts[0].update(labels=1)   # gold
      if len(modals) == 1:
         modals[0].update(labels=1) # gold  
    #else:  
    #  pasts = defragment(nouns, text, original, removed)

  return modals, pasts, verbs, nouns, nounm, nounp

In [ ]:
modals, pasts, verbs, nouns, nounm, nounp = chunk(
  original = 'If another 745 citizens (of 1.5m) had voted for the opposition in Schleswig-Holstein on February 20th, the centre-left coalition would have lost power in the state, which shares a border with Denmark.'
)

print(verbs)
print(nounm)
print(nounp)
print(pasts)
print(modals)     

In [ ]:
import pandas as pd
from tqdm import tqdm
from time import sleep

prefix = '/content/'
test_df = pd.read_csv(prefix + 'train.csv', header=None) # re-name test.csv 
test_df = test_df.drop(index=0)
  
test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1], 
    'labels': ''
})
#print(test_df['id'].count())

i = 0
j = 0
modal_df= pd.DataFrame(columns=['id','labels','text','start', 'end'])
past_df = pd.DataFrame(columns=['id','labels','text','start', 'end'])
simple_df = pd.DataFrame(columns=['id','labels','text'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  oid = test_df['id'][ind]

  modals, pasts, verbs, nouns, nounm, nounp = chunk(original)

  for m in modals:
    m_row = pd.Series(data={
      'id': test_df['id'][ind]+'C'+ str(i),
      'labels': m['labels'],
      'text': m['text'] ,
      'start': m['start'],
      'end': m['end']
    }, name = str(i) )
    modal_df = modal_df.append(m_row)
    i = i+1
 
  for p in pasts:
    p_row = pd.Series(data={
      'id': test_df['id'][ind]+'A'+ str(j),
      'labels': p['labels'],
      'text': p['text'] ,
      'start': p['start'],
      'end': p['end']
    }, name = str(j) )
    past_df = past_df.append(p_row)   
    j = j+1   

  who = str()
  for n in nounp:
    who = who  +  n + ' / '

  what = str()
  for nn in nounm:
    what = what + ' / ' + nn

  i = 0
  j = 0
  n = 0
  for m in modals:
    if len(pasts) > 0:
      for p in pasts:
        labels = str(int(m['labels'])*int(p['labels']))
        simple = who + p['text'] + ' | ' + m['text'] + what
        #print(labels, simple)
        s_row = pd.Series(data={
          'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
          'labels': labels,
          'text': simple
        }, name = str(n) )
        simple_df = simple_df.append(s_row) 
        j = j+1 
        n = n+1
    else:
      s_row = pd.Series(data={
        'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
        'labels':m['labels'],
        'text': who + ' | ' + m['text'] + what
      }, name = str(n) )
      simple_df = simple_df.append(s_row) 
    i = i+1
    n = n+1

  sleep(1)  

simple_df.head(100)

In [ ]:
past_df.to_csv(prefix+'antecedents.tsv', sep='\t', index=False, header=False)
modal_df.to_csv(prefix+'consequents.tsv', sep='\t', index=False, header=False)
simple_df.to_csv(prefix+'simplified.tsv', sep='\t', index=False, header=False)
simple_df.head()

In [ ]:
!mv /content/antecedents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/consequents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/simplified.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/